### **Problem Statement:**

**Objective:** Train a feed-forward neural network using data from the “Appliances Energy Prediction Dataset” and predict the appliance energy consumption for specific dates.

**Note:** The dataset used for this exercise comes from measurements taken from a low-energy home in Stambruges, Belgium, for 5 months (January to May 2016).

#### Un-comment and run the below cell to check list of libraries installed

In [ ]:
# !pip list

#### Un-comment and run the below cell to install tensorflow library

In [ ]:
# !pip install tensorflow

In [ ]:
#Importing Modules

import pandas as pd
import numpy as np
import random as random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

In [ ]:
#Indexes for specific date_times (Do not change these)

index_Jan19_18 = 1160-2
index_Mar12_8 = 8732-2
index_May15_12 = 17972-2

In [ ]:
#Define function to extract data from file, Load training data and convert into Numpy array

def get_data(file_name,test_fraction=0.2):
    df_input = pd.read_csv(file_name,
                           usecols=['lights','T1','RH_1','T2','RH_2','T3','RH_3','T_out','Hour']) #you can modify this - 'RH_out','T3', 'T4','Visibility','Tdewpoint','Day_of_week'
    
    df_target = pd.read_csv(file_name, 
                            usecols=['Appliances'])
    
    print(df_input.head())       #Print first few columns of input data
    print(df_target.head())      #Print first few columns of target data
    
    X_data = df_input.values
    y_data = df_target.values
    
    X_data.astype('float32', copy=False)
    y_data.astype('float32', copy=False)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X_data, 
                                                          y_data, 
                                                          test_size=test_fraction)  #function to split data set
    
    return X_data, y_data, X_train, y_train, X_valid, y_valid # Return training, test data as arrays

In [ ]:
### START YOUR CODE HERE ###

# TO-DO:
# Adjust the parameters: L1, L2,and L3; also the t_fraction based on the values given in the Instruction Document.

L1 = 10 # Number of neurons in Layer 1 (You can change this)
L2 = 5 # Number of neurons in Layer 2 (You can change this)
L3 = 10 # Number of neurons in Layer 3 (You can change this)

# Data set
t_fraction = 0.4 # Fraction of data set used for validation (You can change this)

### END YOUR CODE HERE ###

In [ ]:
# Get training data

file_name = './EE611_W4_Dataset.csv' #NOTE: Change this Path only if the Dataset is in different directory

X_data, y_data, X_train, y_train, X_valid, y_valid = get_data(file_name,
                                                         test_fraction = t_fraction)

print('Shape of training input data set: ', X_train.shape)
print('Shape of training target data set: ', y_train.shape)
print('Number of training samples: ', len(X_train))
print('Shape of validation input data set: ',X_valid.shape)
print('Shape of validation target data set: ',y_valid.shape)

In [ ]:
# Training parameters

epochs = 10           # Number of times to peform training (you can change this)
display_epoch = 2  
learning_rate = 0.01  # The rate at which weights are updated in the neural network
mini_batch_size = 32  # The size to which the training data is split up

In [ ]:
#Placeholders for input and output

Xfeatures = X_train.shape[1]  # Number of features in input data
Yfeatures = 1                 # Number of features in target data

In [ ]:
### START YOUR CODE HERE ###

# TO-DO:
# Comment/Un-comment the Layers (i.e Layer1, Layer2, Layer3) in the Neural network 
# based on the values given in the Instruction Document.

# Building and training a single layer model using Keras
# Reference Link about keras - https://www.tensorflow.org/guide/keras

model = keras.models.Sequential()

#Input Layer
model.add(keras.layers.InputLayer(input_shape=(Xfeatures,),
                                  name='InputLayer'))

#Normalization Layer
model.add(keras.layers.BatchNormalization(axis=1))  #Normalizing values

# Layer1
model.add(keras.layers.Dense(units=L1,
                             activation='relu',
                             bias_initializer='zeros',
                             name='FeedForward1'))  #Add a feed forward layer

# Layer2 
# model.add(keras.layers.Dense(units=L2,
#                              activation='relu',
#                              name='FeedForward2'))  #Add a feed forward layer

# Layer3 
# model.add(keras.layers.Dense(units=L3,
#                              activation='relu',
#                              name='FeedForward3'))  #Add a feed forward layer

#Output layer 
model.add(keras.layers.Dense(units=Yfeatures,
                             name='OutputLayer'))

#Specify loss function and optimizer
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

#Summarize model
model.summary()

### END YOUR CODE HERE ###

In [ ]:
#Run this cell to train model
#Start Training model

model.fit(X_train,
          y_train,
          batch_size=mini_batch_size,
          epochs=epochs,
          validation_data=(X_valid,y_valid),
          shuffle =True,
          verbose=2)

model.save('energy_pred.keras')
print("Model Saved")

In [ ]:
### OPTIONAL ###
### START YOUR CODE HERE ###

prediction = model.predict(X_valid)       #Find predicted value on random data set

### END YOUR CODE HERE ###

In [ ]:
### OPTIONAL ###
### START YOUR CODE HERE ###
# This uses Tensorflow Library to calculate MAE Score
# To know more about MAE check the below links -  
# https://www.tensorflow.org/api_docs/python/tf/keras/metrics/mean_absolute_error
# https://www.educative.io/answers/mean-absolute-error-in-sklearn

mae = tf.keras.losses.MeanAbsoluteError()

print("Mean Absolute Error (MAE):", mae(y_valid, prediction).numpy())

### END YOUR CODE HERE ###

In [ ]:
### OPTIONAL ###
### START YOUR CODE HERE ### 
# This uses sklearn Library to calculate MAE Score (Uncomment if you need to use sklearn)

# from sklearn import metrics

# mae = metrics.mean_absolute_error(y_valid, prediction)
# print("Mean Absolute Error (MAE):", mae)


### END YOUR CODE HERE ###

In [ ]:
#Random Inference session

index = random.randint(0, len(X_valid))                     #Find random input vector from dataset
prediction_rand = model.predict(X_valid, batch_size=1)      #Find predicted value on random data set


print("Predicted load for input vector {} is {} Wh".format(X_valid[index], *prediction_rand[0]))

print("Actual appliance load is {} Wh and Absolute Error is {}".format(y_valid[index][0], 
                                                                       abs(prediction_rand[0]-y_valid[index])[0]))

In [ ]:
### START YOUR CODE HERE ###

# TO-DO:
# Comment/Un-comment the input data corresponding to the dates for which predictions are required 
# based on the values given in the Instruction document.

#Input data for January 19, 18:00, 2016 (Uncomment below lines if this is the date you want)
X_date = np.resize(X_data[index_Jan19_18], (1, Xfeatures))
index = index_Jan19_18

#Input data for March 12, 8:00, 2016 (Uncomment below lines if this is the date you want)
#X_date = np.resize(X_data[index_Mar12_8],(1,Xfeatures))
#index = index_Mar12_8  

#Input data for May 12, 12:00, 2016 (Uncomment below lines if this is the date you want)
# X_date = np.resize(X_data[index_May15_12],(1, Xfeatures) )
# index = index_May15_12

### END YOUR CODE HERE ###

In [ ]:
### START YOUR CODE HERE ###

# TO-DO: 
# 1. Get the prediction value from the first print statement(i.e) *prediction[0] for the corresponding parameters and date
# 2. Get the Absolute Error value from the second print statement(i.e) [abs(prediction[0]-y_data[index])[0])]
# Get the above values and populate them for corresponding row in the Tables.

# Predict for selected date

prediction = model.predict(X_date, batch_size=1)


print("Predicted load for input vector {} is {} Wh".format(X_date[0], *prediction[0]))

print("Actual appliance load is {} Wh and Absolute Error is {}".format(y_data[index][0],
                                                                       abs(prediction[0]-y_data[index])[0]))

### END YOUR CODE HERE ###